In [1]:
import pandas as pd
import numpy as np

tf = pd.read_csv('../data/events_up_to_01062018.csv')


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Divido las fechas**

In [2]:
tf["timestamp"] = pd.to_datetime(tf["timestamp"])
tf["month"] = tf["timestamp"].dt.month
tf["day"] = tf["timestamp"].dt.day
tf["hour"] = tf["timestamp"].dt.hour
tf['dayOfTheWeek'] = tf['timestamp'].dt.dayofweek

** Cantidad de personas que tengo **

In [3]:
#tf.person

In [4]:
persons = tf.person.drop_duplicates()
persons = pd.DataFrame(persons)
persons.count()

person    38829
dtype: int64

**Divido por evento me quedo con las columnas obligatorias por evento**

En en análisis exporatorio me había dado cuenta que no todos los eventos tienen la misma información, por lo tanto, intentaré combinar los eventos con el objetivo de encontrar **nuevos features**. 

Para comenzar divido en distintos DataFrames los distintos eventos, eliminando información que no es útil. 

In [5]:
viewedProduct  = tf[tf['event'] == 'viewed product'].loc[:,['dayOfTheWeek','hour','month','day','person','sku','model','condition','storage','color']]
brandListing = tf[tf['event'] == 'brand listing'].loc[:,['dayOfTheWeek','hour','month','day','person','skus']]
visitedSite = tf[tf['event'] == 'visited site'].loc[:,['dayOfTheWeek','hour','month','day','person','channel','new_vs_returning','city','region','country','device_type',\
                                                       'screen resolution','operating system version','browser version']]
adCampaignHit = tf[tf['event'] == 'ad campaign hit'].loc[:,['dayOfTheWeek','hour','month','day','person','url','campaign_source']]
genericListing = tf[tf['event'] == 'generic listing'].loc[:,['dayOfTheWeek','hour','month','day','person','skus']]
searchedProduct = tf[tf['event'] == 'searched products'].loc[:,['dayOfTheWeek','hour','month','day','person','skus','search_term']]
searchEngineHit = tf[tf['event'] == 'search engine hit'].loc[:,['dayOfTheWeek','hour','month','day','person','search engine']]
checkout = tf[tf['event'] == 'checkout'].loc[:,['dayOfTheWeek','hour','month','day','person','sku','color','storage','model','condition']]
staticPage = tf[tf['event'] == 'staticpage'].loc[:,['dayOfTheWeek','hour','month','day','person','static page']]
conversion = tf[tf['event'] == 'conversion'].loc[:,['dayOfTheWeek','hour','month','day','person','sku','model','color','condition','storage']]
lead = tf[tf['event'] == 'lead'].loc[:,['dayOfTheWeek','hour','month','day','person','model']]

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


**Buscar atributos de otro evento que no sea conversion**


La primera instancia que analizaré es si el usuario es nuevo o no, analizando del visited site el atributo 'new_vs_returning'

### 1º El usuario realiza un compra habiendo visitado la página una vez ###

Intentaré determinar ahora si el usuario que realizó una conversión sólo realizó una única visita al sitio. Para ello debo fijarme en visited site si existe algún valor que sea returning, en tal caso, deja de ser nuevo. 

In [6]:
newVsReturning = visitedSite.loc[:,['person','new_vs_returning']]
newVsReturning = newVsReturning.groupby('person').new_vs_returning.value_counts().to_frame('cantidad')
newVsReturning = newVsReturning.unstack().fillna(0)
newVsReturning = newVsReturning.reset_index()
newVsReturning.columns = ['person','new','returning']
newVsReturning['esNuevo'] = newVsReturning['returning'] == 0

Agrego este atributo a conversion con un merge

In [7]:
labels_f = pd.merge(persons, newVsReturning.loc[:,['esNuevo','person']], how = 'left', on = 'person')

In [8]:
labels_f.esNuevo.isnull().value_counts()

False    38242
True       587
Name: esNuevo, dtype: int64

In [9]:
moda = labels_f.esNuevo.mode().iloc[0]
print(type(moda))
labels_f['esNuevo'] = labels_f['esNuevo'].fillna(moda)


<class 'bool'>


In [10]:
labels_f

,person,esNuevo
0,4886f805,True
1,ad93850f,False
2,0297fc1e,False
3,2d681dd8,False
4,cccea85e,False
5,4c8a8b93,False
6,1b9f7cf6,False
7,29ebb414,False
8,de8fe91b,False
9,45baf068,True


### 2º Cantidad de visitas del usuario a la página ###

Analizaré ahora la frecuencia de entradas al sitio de un determinado usuario. Para ello analizaré la relación entre new y returnings (le sumo 1 a la cantidad de returnings). 

In [11]:
newVsReturning['cantidadDeVisitas'] = (newVsReturning['new'] + newVsReturning['returning']).astype('float32')
labels_f = pd.merge(labels_f, newVsReturning.loc[:,['cantidadDeVisitas','person']],how = 'left', on = 'person')

In [12]:
labels_f['cantidadDeVisitas'] = labels_f['cantidadDeVisitas'].astype('float32')
labels_f['cantidadDeVisitas'] = labels_f['cantidadDeVisitas'].fillna(labels_f['cantidadDeVisitas'].mean())

### 3º El usuario ingresó a la página por algún aviso publicitario alguna vez ###

Analizaré si el usario ingresó a la página desde una publicidad (en algún momento) para ello usaré el evento ad_campaing_hit.

In [13]:
personasAdHit = adCampaignHit.loc[:,['person']]
personasAdHit['ingresoPorPublicidad'] = True
personasAdHit = personasAdHit.drop_duplicates()

In [14]:
labels_f = pd.merge(labels_f, personasAdHit, on = 'person', how = 'left')


In [15]:
labels_f['ingresoPorPublicidad'].isnull().value_counts()

False    31900
True      6929
Name: ingresoPorPublicidad, dtype: int64

In [16]:
labels_f['ingresoPorPublicidad'] = labels_f.fillna(False)

In [17]:
labels_f['ingresoPorPublicidad'].isnull().value_counts()

False    38829
Name: ingresoPorPublicidad, dtype: int64

In [18]:
labels_f.esNuevo.isnull().value_counts()

False    38829
Name: esNuevo, dtype: int64

In [92]:
labels_f.ingresoPorPublicidad

0        4886f805
1        ad93850f
2        0297fc1e
3        2d681dd8
4        cccea85e
5        4c8a8b93
6        1b9f7cf6
7        29ebb414
8        de8fe91b
9        45baf068
10       99abca5a
11       3dc1950f
12       8ea4c165
13       64f45e8d
14       d8cfe234
15       ade6b3bf
16       d73ae9ef
17       d6bc64df
18       ec9c4059
19       b2cde41a
20       15ea8012
21       43790d8f
22       d21b8e6b
23       2a724d87
24       429c8732
25       686c49c9
26       4e03ed9a
27       a08d42ea
28       16fad36a
29       be32e904
           ...   
38799    154d2935
38800    3adf7ca4
38801    c9552fad
38802    2e89874a
38803    ef4e52ab
38804    100f33c8
38805    90251b6a
38806    85e0f62a
38807    0afaecc8
38808    2c209f87
38809    4ddb8c19
38810    9979305a
38811    25bd8078
38812    87d306fc
38813    a2b1e355
38814    fb88a7ea
38815    1315ce31
38816    af3374ad
38817    023581f9
38818    9707cd0e
38819    88b91c5d
38820    6f7632db
38821    a1c2a901
38822    ed3f80d7
38823    2

Supongo que la información que no tengo en ad_campaing_hit de los usuarios, es porque no ingresaron por una campaña publicitaria alguna vez a la página y por lo tanto **este atributo puede usarse para lo 19414 labels**. 

### 4º Campaña publicitaria por la que ingresó el usuario ###

Analizaré por qué campaña ingresaron los usuarios (en algún momento y luego realizaron una conversión), agregando esa información de ad_campaing_hit

In [20]:
campaniaPersonasAdHit = adCampaignHit.loc[:,['person','campaign_source']]
campaniaPersonasAdHit = campaniaPersonasAdHit.groupby('person').campaign_source.value_counts().to_frame('cantidad')
campaniaPersonasAdHit = campaniaPersonasAdHit.unstack().fillna(0).reset_index()
campaniaPersonasAdHit.head()
campaniaPersonasAdHit.columns=['person','cant_facebook','cant_facebookAds', 'cant_facebookSocial', 'cant_marketing_social', 'cant_afiliado', 'cant_afilio', 'cant_bing', 'cant_blog','cant_buscape', 'cant_criteo', 'cant_datacrush', 'cant_emblue', 'cant_gizmodo', 'cant_google', 'cant_indexa', 'cant_manifest', 'cant_mercadopago', 'cant_onsite', 'cant_rakuten','cant_rtbhouse', 'cant_voxus', 'cant_yotpo', 'cant_zanox']

**FALTA COMPLETAR ESTO, SON MUCHAS CAMPAÑAS**

### 5º información sobre el dispositivo por el cual se conectaron ###

El único evento que posee información sobre el dispositivo al cual se conecto es Visited Site. Se registra un Visited Site cada vez que el usuario ingresa a Trocafone (cualquiera sea el motivo). Repetiré el análisis hecho en el análisis exploratorio en el que mostrábamos que la mayoría de los usuarios se conectaban por un sólo dispositivo.


In [21]:
def contar_columnas(x):#(col1,col2,col3,col4):
    t=0
    if x['tablet']>0:
        t += 1
    if x['smartphone']>0:
        t += 1
    if x['computer']>0:
        t += 1
    if x['unknown']>0:
        t += 1
    return t


#print(visited_site.device_type.value_counts()) -> LOS UNKNOWN SON DESPRECIABLES. 
dispositivos= visitedSite.groupby('person')['device_type'].value_counts().to_frame('cantidad')
dispositivos = dispositivos.unstack().fillna(0).reset_index()
dispositivos.columns = ['person','computer','smartphone','tablet','unknown']
dispositivos = dispositivos.set_index('person')
dispositivos['porcentaje'] = dispositivos.apply(lambda p:contar_columnas(p), axis = 1)


print(dispositivos['porcentaje'].value_counts(normalize = True).to_frame())

#quito aquellos que se hayan conectado por más de un dispositivo
dispositivos = dispositivos[dispositivos['porcentaje'] <= 1]
visited_device = visitedSite.loc[:,['person','device_type']].drop_duplicates()
dispositivos = dispositivos.reset_index()
visited_device = pd.merge(dispositivos.loc[:,['person']],visited_device, on= 'person',how = 'left').drop_duplicates()


   porcentaje
1    0.959547
2    0.039721
3    0.000732


Por lo tanto existe un 96 porciento de los usuarios que se conectaron de un único tipo de dispositivo, por lo tanto, despreciaremos el otro 4 porciento. 


In [22]:
labels_f = pd.merge(labels_f, visited_device, how = 'left', on = 'person')
#le cambio el nombre a la columna device type para que no se confunda con lo que compraron

In [23]:
labels_f.device_type.isnull().value_counts()

False    36695
True      2134
Name: device_type, dtype: int64

In [24]:
labels_f['device_type'] = labels_f['device_type'].fillna(labels_f.device_type.mode().iloc[0])

In [25]:
labels_f.device_type.isnull().value_counts()

False    38829
Name: device_type, dtype: int64

In [26]:
labels_f['deviceSmartphone'] = labels_f['device_type'] == 'smartphone'
labels_f['deviceComputer'] = labels_f['device_type'] == 'computer'
labels_f['deviceTablet'] = labels_f['device_type'] == 'tablet'
labels_f['deviceUnknown'] = labels_f['device_type'] == 'unknown'

Es decir que **de los 19414 labels el atributo device_type presenta información para 18370** entonces 1044 labels no aportan este tipo de información. 

### 6º Cantidad de veces que el usuario vio un producto ###

Analizaré ahora la cantidad de veces que el usuario vio un producto 

In [27]:
cantidadDeVisitasAProductos = viewedProduct.loc[:,['person']]
cantidadDeVisitasAProductos['cantVisitasAVP'] = 1
cantidadDeVisitasAProductos = cantidadDeVisitasAProductos.groupby('person').cantVisitasAVP.sum().to_frame()


In [28]:
labels_f = pd.merge(labels_f, cantidadDeVisitasAProductos, on= 'person', how = 'left')

In [29]:
labels_f.cantVisitasAVP.isnull().value_counts()

False    37130
True      1699
Name: cantVisitasAVP, dtype: int64

Supongo que como no tenemos información de 1699 labels sobre la cantidad de visitas a productos estos nunca lo hicieron.

In [30]:
personasSinVisitas = labels_f.loc[:,['person','cantVisitasAVP']]
personasSinVisitas = personasSinVisitas.fillna(0)

In [31]:
labels_f = pd.merge(labels_f, personasSinVisitas, how = 'left', on = 'person')

In [32]:
labels_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Data columns (total 11 columns):
person                  38829 non-null object
esNuevo                 38829 non-null bool
cantidadDeVisitas       38829 non-null float32
ingresoPorPublicidad    38829 non-null object
device_type             38829 non-null object
deviceSmartphone        38829 non-null bool
deviceComputer          38829 non-null bool
deviceTablet            38829 non-null bool
deviceUnknown           38829 non-null bool
cantVisitasAVP_x        37130 non-null float64
cantVisitasAVP_y        38829 non-null float64
dtypes: bool(5), float32(1), float64(2), object(3)
memory usage: 2.1+ MB


In [33]:
labels_f = labels_f.loc[:,['person','label','esNuevo','cantidadDeVisitas','ingresoPorPublicidad','deviceSmartphone','deviceComputer','deviceTablet','deviceUnknown','cantVisitasAVP_y']]
labels_f.columns = ['person','label','esNuevo','cantidadDeVisitas','ingresoPorPublicidad','deviceSmartphone','deviceComputer','deviceTablet','deviceUnknown','cantVisitasAVP']

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [34]:
labels_f.cantVisitasAVP.isnull().value_counts()

False    38829
Name: cantVisitasAVP, dtype: int64

### 7º Momento del día en el que realizan las visitas ###

Usando visitedSite analizaré el momento del día en el que el usuario realizó las visitas. Lo agruparé en las siguientes clases:

-Mañana: si el usuario realizó la visita entre las 7 y 12 hs. 

-Tarde: si el usuario realizó la visita entre las 12 y las 18 hs. 

-Noche: si el usuario realizó la visita entre las 18 y 24 hs. 

-Madrugada: si el usuario realizó la visita entre las 24 y 7 hs. 

In [35]:
momentoDeVisita = visitedSite.loc[:,['person','hour']]
momentoDeVisita['maniana'] = ((momentoDeVisita['hour'] < 12 ) & (momentoDeVisita['hour'] >= 7)).astype('int32')
momentoDeVisita['tarde'] = ((momentoDeVisita['hour'] < 18 ) & (momentoDeVisita['hour'] >= 12)).astype('int32')
momentoDeVisita['noche'] = ((momentoDeVisita['hour'] < 24 ) & (momentoDeVisita['hour'] >= 18)).astype('int32')
momentoDeVisita['madrugada'] = ((momentoDeVisita['hour'] < 7 ) & (momentoDeVisita['hour'] >= 24)).astype('int32')
momentoDeVisita = momentoDeVisita.groupby('person').sum().reset_index()
momentoDeVisita = momentoDeVisita.loc[:,['person','maniana','tarde','noche','madrugada']]

In [36]:
labels_f = pd.merge(labels_f,momentoDeVisita, on = 'person', how = 'left') 

In [37]:
labels_f['maniana'].isnull().value_counts()

False    38242
True       587
Name: maniana, dtype: int64

Por lo tanto, **los atributos: maniana, tarde, noche y madrugada no fueron asignados para 288 labels**. Esto concuerda con el atributo device type obtenido anterior mentde del mismo evento: visited site.

In [38]:
labels_f['maniana'] = labels_f['maniana'].fillna(labels_f['maniana'].median())
labels_f['tarde'] = labels_f['tarde'].fillna(labels_f['tarde'].median())
labels_f['noche'] = labels_f['noche'].fillna(labels_f['noche'].median())
labels_f['madrugada'] = labels_f['madrugada'].fillna(labels_f['madrugada'].median())

In [39]:
labels_f.maniana.isnull().value_counts()

False    38829
Name: maniana, dtype: int64

### 8º Cantidad de visitas por canal que realiza el usuario###

Analizaré el canal por el que el usuario realiza la visita

In [40]:
canalDeVisita = visitedSite.loc[:,['person','channel']]
canalDeVisita = canalDeVisita.groupby('person').channel.value_counts().to_frame('cantidad')
canalDeVisita = canalDeVisita.unstack()
canalDeVisita = canalDeVisita.reset_index()
canalDeVisita.columns = ['person','cantDirect','cantEmail','cantOrganic','cantPaid','cantReferral',\
                           'cantSocial','cantUnknown']
canalDeVisita = canalDeVisita.fillna(0)

In [41]:
labels_f = pd.merge(labels_f,canalDeVisita,on = 'person', how = 'left')

In [42]:
labels_f['cantDirect'].isnull().value_counts()

False    38242
True       587
Name: cantDirect, dtype: int64

In [43]:
labels_f['cantDirect'] = labels_f['cantDirect'].fillna(labels_f['cantDirect'].median())
labels_f['cantEmail'] = labels_f['cantEmail'].fillna(labels_f['cantEmail'].median())
labels_f['cantOrganic'] = labels_f['cantOrganic'].fillna(labels_f['cantOrganic'].median())
labels_f['cantPaid'] = labels_f['cantPaid'].fillna(labels_f['cantPaid'].median())
labels_f['cantReferral'] = labels_f['cantReferral'].fillna(labels_f['cantReferral'].median())
labels_f['cantSocial'] = labels_f['cantSocial'].fillna(labels_f['cantSocial'].median())
labels_f['cantUnknown'] = labels_f['cantUnknown'].fillna(labels_f['cantUnknown'].median())

Por lo tanto, **los atributos: cantDirect, cantEmail, cantOrganic, cantPaid, cant referral, cantSocial, cantUnknown que refieren a la cantidad de accesos hecho por cada canal, no tiene fueron asignados para 288 labels**. Esto concuerda con todos los atributos obteneidos del mismo evento: vistedSite

In [44]:
labels_f.cantDirect.isnull().value_counts()

False    38829
Name: cantDirect, dtype: int64

### 9º Cantidad de visitas por día de la semana ###

Analizaré la cantidad de visitas que el usuario realiza por día de la semana. Usando el evento visited site. 

In [45]:
diaSemana = visitedSite.loc[:,['person','dayOfTheWeek']]
diaSemana = diaSemana.groupby('person').dayOfTheWeek.value_counts().to_frame('cantidad').unstack()

In [46]:
diaSemana.head()

cantidad                               
dayOfTheWeek        0    1    2    3    4    5     6
person                                              
0008ed71          NaN  NaN  NaN  2.0  NaN  NaN   NaN
00091926          2.0  4.0  3.0  6.0  3.0  5.0  11.0
00091a7a          1.0  NaN  NaN  NaN  NaN  NaN   NaN
000ba417          NaN  NaN  NaN  4.0  NaN  2.0   NaN
000c79fe          NaN  1.0  NaN  NaN  NaN  NaN   NaN

Supongo que los usuarios que no tiene información ese día de la semana es pq no visitaron la página en ese día de la semana. 

In [47]:
diaSemana = diaSemana.fillna(0)

In [48]:
diaSemana = diaSemana.reset_index()

In [49]:
diaSemana.columns = ['person','cantVisitasDomingo','cantVisitasLunes',\
                    'cantVisitasMartes', 'cantVisitasMiercoles', \
                    'cantVisitasJueves', 'cantVisitasViernes', \
                    'cantVisitasSabado']

In [50]:
labels_f = pd.merge(labels_f,diaSemana, how = 'left', on = 'person')

In [51]:
labels_f['cantVisitasMartes'].isnull().value_counts() 

False    38242
True       587
Name: cantVisitasMartes, dtype: int64

In [52]:
labels_f['cantVisitasDomingo'] = labels_f['cantVisitasDomingo'].fillna(labels_f['cantVisitasDomingo'].median())
labels_f['cantVisitasLunes'] = labels_f['cantVisitasLunes'].fillna(labels_f['cantVisitasLunes'].median())
labels_f['cantVisitasMartes'] = labels_f['cantVisitasMartes'].fillna(labels_f['cantVisitasMartes'].median())
labels_f['cantVisitasMiercoles'] = labels_f['cantVisitasMiercoles'].fillna(labels_f['cantVisitasMiercoles'].median())
labels_f['cantVisitasJueves'] = labels_f['cantVisitasJueves'].fillna(labels_f['cantVisitasJueves'].median())
labels_f['cantVisitasViernes'] = labels_f['cantVisitasViernes'].fillna(labels_f['cantVisitasViernes'].median())
labels_f['cantVisitasSabado'] = labels_f['cantVisitasSabado'].fillna(labels_f['cantVisitasSabado'].median())

In [53]:
labels_f.cantVisitasDomingo.isnull().value_counts()

False    38829
Name: cantVisitasDomingo, dtype: int64

### 10º cantidad de eventos que tiene registrado cada usuario ###

In [54]:
cantidadDeEventos = tf.loc[:,['event','person']]
cantidadDeEventos = cantidadDeEventos.loc[cantidadDeEventos['event'] != 'conversion']
#no tiene sentido este
cantidadDeEventos = cantidadDeEventos.groupby('person').event.count().to_frame()
cantidadDeEventos = cantidadDeEventos.reset_index()
cantidadDeEventos.columns = ['person','cantidadDeEventos']

In [55]:
labels_f = pd.merge(labels_f, cantidadDeEventos, on = 'person', how = 'left' )


In [56]:
labels_f.cantidadDeEventos.isnull().value_counts()

False    38827
True         2
Name: cantidadDeEventos, dtype: int64

In [57]:
mediana = labels_f.cantidadDeEventos.median()
labels_f['cantidadDeEventos'] =labels_f['cantidadDeEventos'].fillna(mediana)

In [58]:
labels_f.cantidadDeEventos.isnull().value_counts()

False    38829
Name: cantidadDeEventos, dtype: int64

### 11º Cantidad de eventos, discretizado por tipo de evento, que tiene cada usuario ###

In [59]:
cantidadDeEventos = tf.loc[:,['event','person']]
cantidadDeEventos = cantidadDeEventos.loc[cantidadDeEventos['event'] != 'conversion']
#no tiene sentido este
cantidadDeEventos = cantidadDeEventos.groupby('person').event.value_counts().to_frame().unstack()
cantidadDeEventos = cantidadDeEventos.reset_index()
cantidadDeEventos.columns = ['person','cantAdCampaignHit', 'cantBandListing',\
                             'cantCheckout','cantGenericListing','cantLead',\
                            'cantSearchEngineHit', 'cantSearchedProducts',\
                             'cantStaticpage', 'cantViewedProduct', 'cantVisitedSite']

In [60]:
cantidadDeEventos.head()

,person,cantAdCampaignHit,cantBandListing,cantCheckout,cantGenericListing,cantLead,cantSearchEngineHit,cantSearchedProducts,cantStaticpage,cantViewedProduct,cantVisitedSite
0,0008ed71,NaN,NaN,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0
1,00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,372.0,34.0
2,00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
3,000ba417,1.0,24.0,6.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
4,000c79fe,1.0,NaN,1.0,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [61]:
cantidadDeEventos['cantAdCampaignHit'] = cantidadDeEventos['cantAdCampaignHit'].fillna(cantidadDeEventos['cantAdCampaignHit'].median())
cantidadDeEventos['cantBandListing'] = cantidadDeEventos['cantBandListing'].fillna(cantidadDeEventos['cantBandListing'].median())
cantidadDeEventos['cantCheckout'] = cantidadDeEventos['cantCheckout'].fillna(cantidadDeEventos['cantCheckout'].median())
cantidadDeEventos['cantGenericListing'] = cantidadDeEventos['cantGenericListing'].fillna(cantidadDeEventos['cantGenericListing'].median())
cantidadDeEventos['cantLead'] = cantidadDeEventos['cantLead'].fillna(cantidadDeEventos['cantLead'].median())
cantidadDeEventos['cantSearchEngineHit'] = cantidadDeEventos['cantSearchEngineHit'].fillna(cantidadDeEventos['cantSearchEngineHit'].median())
cantidadDeEventos['cantSearchedProducts'] = cantidadDeEventos['cantSearchedProducts'].fillna(cantidadDeEventos['cantSearchedProducts'].median())
cantidadDeEventos['cantStaticpage'] = cantidadDeEventos['cantStaticpage'].fillna(cantidadDeEventos['cantStaticpage'].median())
cantidadDeEventos['cantViewedProduct'] = cantidadDeEventos['cantViewedProduct'].fillna(cantidadDeEventos['cantViewedProduct'].median())
cantidadDeEventos['cantVisitedSite'] = cantidadDeEventos['cantVisitedSite'].fillna(cantidadDeEventos['cantVisitedSite'].median())



In [62]:
labels_f = pd.merge(labels_f,cantidadDeEventos,on = 'person', how = 'left')

In [63]:
labels_f['cantAdCampaignHit'] = labels_f['cantAdCampaignHit'].fillna(labels_f['cantAdCampaignHit'].median())
labels_f['cantBandListing'] = labels_f['cantBandListing'].fillna(labels_f['cantBandListing'].median())
labels_f['cantCheckout'] = labels_f['cantCheckout'].fillna(labels_f['cantCheckout'].median())
labels_f['cantGenericListing'] = labels_f['cantGenericListing'].fillna(labels_f['cantGenericListing'].median())
labels_f['cantLead'] = labels_f['cantLead'].fillna(labels_f['cantLead'].median())
labels_f['cantSearchEngineHit'] = labels_f['cantSearchEngineHit'].fillna(labels_f['cantSearchEngineHit'].median())
labels_f['cantSearchedProducts'] = labels_f['cantSearchedProducts'].fillna(labels_f['cantSearchedProducts'].median())
labels_f['cantStaticpage'] = labels_f['cantStaticpage'].fillna(labels_f['cantStaticpage'].median())
labels_f['cantViewedProduct'] = labels_f['cantViewedProduct'].fillna(labels_f['cantViewedProduct'].median())
labels_f['cantVisitedSite'] = labels_f['cantVisitedSite'].fillna(labels_f['cantVisitedSite'].median())



In [64]:
labels_f.cantAdCampaignHit.isnull().value_counts()

False    38829
Name: cantAdCampaignHit, dtype: int64

### Agrego features según lo anterior pero en vez de la cantidad indico si lo hizo o no ###

### 12º El usuario vio un producto ### 

In [65]:
visitoUnProducto = viewedProduct.loc[:,['person']]
visitoUnProducto['cantVisitasAVP'] = 1
visitoUnProducto = visitoUnProducto.groupby('person').cantVisitasAVP.sum().to_frame()

In [66]:
visitoUnProducto['visitoUnProducto'] = visitoUnProducto.cantVisitasAVP > 0
visitoUnProducto = visitoUnProducto.reset_index()
visitoUnProducto = visitoUnProducto.loc[:,['person','visitoUnProducto']]

In [67]:
labels_f = pd.merge(labels_f,visitoUnProducto, on = 'person', how = 'left')

In [68]:
labels_f.visitoUnProducto.isnull().value_counts()

False    37130
True      1699
Name: visitoUnProducto, dtype: int64

**Como todos los usuarios visitaron un producto este feature no sirve.**

Lo que suscede es que esos 37130 son True y los demás son Nan y si completo con la moda no tiene sentido. 

In [69]:
labels_f = labels_f.loc[:,['person', 'label', 'esNuevo', 'cantidadDeVisitas',
       'ingresoPorPublicidad', 'deviceSmartphone', 'deviceComputer',
       'deviceTablet', 'deviceUnknown', 'cantVisitasAVP', 'maniana', 'tarde',
       'noche', 'madrugada', 'cantDirect', 'cantEmail', 'cantOrganic',
       'cantPaid', 'cantReferral', 'cantSocial', 'cantUnknown',
       'cantVisitasDomingo', 'cantVisitasLunes', 'cantVisitasMartes',
       'cantVisitasMiercoles', 'cantVisitasJueves', 'cantVisitasViernes',
       'cantVisitasSabado', 'cantidadDeEventos', 'cantAdCampaignHit',
       'cantBandListing', 'cantCheckout', 'cantGenericListing', 'cantLead',
       'cantSearchEngineHit', 'cantSearchedProducts', 'cantStaticpage',
       'cantViewedProduct', 'cantVisitedSite']]

### 13º El usuario visita la página en un momento determinado del día ###

In [70]:
momentoDeVisita_2 = momentoDeVisita
momentoDeVisita_2['visitaALaManiana'] = momentoDeVisita['maniana']>0
momentoDeVisita_2['visitaALaTarde'] = momentoDeVisita['tarde']>0
momentoDeVisita_2['visitaALaNoche'] = momentoDeVisita['noche']>0
momentoDeVisita_2['visitaALaMadrugada'] = momentoDeVisita['madrugada']>0
momentoDeVisita_2 = momentoDeVisita_2.loc[:,['person','visitaALaManiana','visitaALaNoche',\
                                         'visitaALaTarde', 'visitaALaMadrugada']]

Supongo que aquellos usuarios que no tienen información es porque no visitaron en ese momento. 

In [71]:
momentoDeVisita_2 = momentoDeVisita_2.fillna(False)

In [72]:
labels_f = pd.merge(labels_f, momentoDeVisita_2, on = 'person', how = 'left')


In [73]:
labels_f['visitaALaManiana'] = labels_f['visitaALaManiana'].fillna(labels_f['visitaALaManiana'].mode().iloc[0])
labels_f['visitaALaTarde'] = labels_f['visitaALaTarde'].fillna(labels_f['visitaALaTarde'].mode().iloc[0])
labels_f['visitaALaNoche'] = labels_f['visitaALaNoche'].fillna(labels_f['visitaALaNoche'].mode().iloc[0])
labels_f['visitaALaMadrugada'] = labels_f['visitaALaMadrugada'].fillna(labels_f['visitaALaMadrugada'].mode().iloc[0])

In [74]:
labels_f.visitaALaManiana.isnull().value_counts()

False    38829
Name: visitaALaManiana, dtype: int64

### 14º el usuario visitó la página por determinado canal ###

In [75]:
canalDeVisita_2 = canalDeVisita
canalDeVisita_2['canalDirect'] = canalDeVisita['cantDirect'] >0 
canalDeVisita_2['canalEmail'] = canalDeVisita['cantEmail'] >0 
canalDeVisita_2['canalOrganic'] = canalDeVisita['cantOrganic'] >0 
canalDeVisita_2['canalPaid'] = canalDeVisita['cantPaid'] >0 
canalDeVisita_2['canalRefferal'] = canalDeVisita['cantReferral'] >0 
canalDeVisita_2['canalSocial'] = canalDeVisita['cantUnknown'] > 0

canalDeVisita_2 = canalDeVisita_2.loc[:,['person','canalDirect', 'canalEmail','canalOrganic',\
                                    'canalPaid','canalRefferal', 'canalSocial']]

In [76]:
labels_f = pd.merge(labels_f, canalDeVisita_2, on = 'person', how = 'left')

In [77]:
labels_f['canalDirect'] = labels_f['cantDirect'].fillna(labels_f['cantDirect'].mode().iloc[0]) 
labels_f['canalEmail'] = labels_f['cantEmail'].fillna(labels_f['cantEmail'].mode().iloc[0]) 
labels_f['canalOrganic'] = labels_f['cantOrganic'].fillna(labels_f['cantOrganic'].mode().iloc[0]) 
labels_f['canalPaid'] = labels_f['cantPaid'].fillna(labels_f['cantPaid'].mode().iloc[0]) 
labels_f['canalRefferal'] = labels_f['cantReferral'].fillna(labels_f['cantReferral'].mode().iloc[0]) 
labels_f['canalSocial'] = labels_f['cantUnknown'].fillna(labels_f['cantUnknown'].mode().iloc[0])

In [78]:
labels_f.canalDirect.isnull().value_counts()

False    38829
Name: canalDirect, dtype: int64

### 15º el usuario realizó determinado evento### 

In [82]:
cantidadDeEventos_2 = cantidadDeEventos
cantidadDeEventos_2['adCampaignHit'] = cantidadDeEventos['cantAdCampaignHit']>0
cantidadDeEventos_2['bandListing'] = cantidadDeEventos['cantBandListing']>0
cantidadDeEventos_2['checkout'] = cantidadDeEventos['cantCheckout']>0
cantidadDeEventos_2['genericListing'] = cantidadDeEventos['cantGenericListing']>0
cantidadDeEventos_2['searchEngineHit'] = cantidadDeEventos['cantSearchEngineHit']>0
cantidadDeEventos_2['searchedProducts'] = cantidadDeEventos['cantSearchedProducts']>0
cantidadDeEventos_2['staticpage'] = cantidadDeEventos['cantStaticpage']>0
cantidadDeEventos_2['viewedProduct'] = cantidadDeEventos['cantViewedProduct']>0
cantidadDeEventos_2['visitedSite'] = cantidadDeEventos['cantVisitedSite']>0
cantidadDeEventos_2['lead'] = cantidadDeEventos['cantLead']>0

In [83]:
cantidadDeEventos_2 = cantidadDeEventos_2.loc[:,['person','adCampaignHit','bandListing','checkout','genericListing','searchEngineHit','searchedProducts','staticpage','viewedProduct','visitedSite','lead']]

In [84]:
labels_f = pd.merge(labels_f,cantidadDeEventos_2, on = 'person', how = 'left')

In [86]:
labels_f['adCampaignHit'] = labels_f['adCampaignHit'].fillna(labels_f['adCampaignHit'].mode().iloc[0])
labels_f['bandListing'] = labels_f['bandListing'].fillna(labels_f['bandListing'].mode().iloc[0])
labels_f['checkout'] = labels_f['checkout'].fillna(labels_f['checkout'].mode().iloc[0])
labels_f['genericListing'] = labels_f['genericListing'].fillna(labels_f['genericListing'].mode().iloc[0])
labels_f['lead'] = labels_f['lead'].fillna(labels_f['lead'].median())
labels_f['searchEngineHit'] = labels_f['searchEngineHit'].fillna(labels_f['searchEngineHit'].mode().iloc[0])
labels_f['searchedProducts'] = labels_f['searchedProducts'].fillna(labels_f['searchedProducts'].mode().iloc[0])
labels_f['staticpage'] = labels_f['staticpage'].fillna(labels_f['staticpage'].mode().iloc[0])
labels_f['viewedProduct'] = labels_f['viewedProduct'].fillna(labels_f['viewedProduct'].mode().iloc[0])
labels_f['visitedSite'] = labels_f['visitedSite'].fillna(labels_f['visitedSite'].mode().iloc[0])

In [87]:
labels_f.checkout.isnull().value_counts()

False    38829
Name: checkout, dtype: int64

Index(['person', 'label', 'esNuevo', 'cantidadDeVisitas',
       'ingresoPorPublicidad', 'deviceSmartphone', 'deviceComputer',
       'deviceTablet', 'deviceUnknown', 'cantVisitasAVP', 'maniana', 'tarde',
       'noche', 'madrugada', 'cantDirect', 'cantEmail', 'cantOrganic',
       'cantPaid', 'cantReferral', 'cantSocial', 'cantUnknown',
       'cantVisitasDomingo', 'cantVisitasLunes', 'cantVisitasMartes',
       'cantVisitasMiercoles', 'cantVisitasJueves', 'cantVisitasViernes',
       'cantVisitasSabado', 'cantidadDeEventos', 'cantAdCampaignHit',
       'cantBandListing', 'cantCheckout', 'cantGenericListing', 'cantLead',
       'cantSearchEngineHit', 'cantSearchedProducts', 'cantStaticpage',
       'cantViewedProduct', 'cantVisitedSite', 'visitaALaManiana',
       'visitaALaNoche', 'visitaALaTarde', 'visitaALaMadrugada', 'canalDirect',
       'canalEmail', 'canalOrganic', 'canalPaid', 'canalRefferal',
       'canalSocial', 'adCampaignHit', 'bandListing', 'checkout',
       'genericLis

### 16º el usuario visitó los productos más populares ###

In [68]:
productosMasVistos = viewedProduct['model'].value_counts().reset_index().iloc[0:10,0].tolist()
personasYProductosMasVistos = viewedProduct.loc[:,['model','person']]
personasYProductosMasVistos['vioUnProductoMasVisto'] = personasYProductosMasVistos['model'].str in productosMasVistos

personasYProductosMasVistos.vioUnProductoMasVisto.value_counts()
productosMasVistos

['iPhone 6',
 'iPhone 5s',
 'iPhone 6S',
 'iPhone 7',
 'Samsung Galaxy S7 Edge',
 'iPhone 7 Plus',
 'Samsung Galaxy S7',
 'iPhone 6S Plus',
 'iPhone SE',
 'Samsung Galaxy S8']

In [69]:
labels_f['cantidadDeVisitas'] = labels_f['cantidadDeVisitas'].astype('float64')

In [70]:
labels_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38242 entries, 0 to 38241
Data columns (total 58 columns):
person                  38242 non-null object
label                   0 non-null float64
esNuevo                 38242 non-null bool
cantidadDeVisitas       38242 non-null float64
ingresoPorPublicidad    38242 non-null object
deviceSmartphone        38242 non-null bool
deviceComputer          38242 non-null bool
deviceTablet            38242 non-null bool
deviceUnknown           38242 non-null bool
cantVisitasAVP          38242 non-null float64
maniana                 38242 non-null float64
tarde                   38242 non-null float64
noche                   38242 non-null float64
madrugada               38242 non-null float64
cantDirect              38242 non-null float64
cantEmail               38242 non-null float64
cantOrganic             38242 non-null float64
cantPaid                38242 non-null float64
cantReferral            38242 non-null float64
cantSocial         

In [71]:
labels_f.columns

Index(['person', 'label', 'esNuevo', 'cantidadDeVisitas',
       'ingresoPorPublicidad', 'deviceSmartphone', 'deviceComputer',
       'deviceTablet', 'deviceUnknown', 'cantVisitasAVP', 'maniana', 'tarde',
       'noche', 'madrugada', 'cantDirect', 'cantEmail', 'cantOrganic',
       'cantPaid', 'cantReferral', 'cantSocial', 'cantUnknown',
       'cantVisitasDomingo', 'cantVisitasLunes', 'cantVisitasMartes',
       'cantVisitasMiercoles', 'cantVisitasJueves', 'cantVisitasViernes',
       'cantVisitasSabado', 'cantidadDeEventos', 'cantAdCampaignHit',
       'cantBandListing', 'cantCheckout', 'cantGenericListing', 'cantLead',
       'cantSearchEngineHit', 'cantSearchedProducts', 'cantStaticpage',
       'cantViewedProduct', 'cantVisitedSite', 'visitaALaMañana',
       'visitaALaNoche', 'visitaALaTarde', 'visitaALaMadrugada', 'canalDirect',
       'canalEmail', 'canalOrganic', 'canalPaid', 'canalRefferal',
       'canalSocial', 'adCampaignHit', 'bandListing', 'checkout',
       'genericList

In [91]:
labels_f.to_csv('featuresMagui.csv')